In [172]:
import lightkurve as lk
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, Concatenate, Conv1D, MaxPool1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

# Metadata

## Načtení

In [173]:
pos = pd.read_csv("positive.csv")
pos

,host,period
0,Kepler-1,2.470
1,Kepler-2,2.200
2,Kepler-3,4.890
3,Kepler-4,3.210
4,Kepler-5,3.580
5,Kepler-6,3.230
6,Kepler-7,4.890
7,Kepler-8,3.522
8,Kepler-11,32.000
9,Kepler-11,22.680


In [174]:
neg = pd.read_csv("negative.csv")
neg

,host,period
0,KIC 10000785,1.00
1,KIC 10000785,6.00
2,KIC 10000785,36.00
3,KIC 10000785,85.00
4,KIC 10000797,1.00
5,KIC 10000797,6.00
6,KIC 10000797,36.00
7,KIC 10000797,85.00
8,KIC 10000800,1.00
9,KIC 10000800,6.00


## Seskupení dle systémů

In [175]:
group = lambda df: df.groupby("host").period.apply(list).to_dict()
grouped_pos = group(pos)
grouped_pos

{'Kepler-1': [2.47],
 'Kepler-11': [32.0, 22.68, 13.02],
 'Kepler-12': [4.44],
 'Kepler-13': [1.76],
 'Kepler-14': [6.79],
 'Kepler-15': [4.94],
 'Kepler-17': [1.48],
 'Kepler-18': [7.65, 14.86],
 'Kepler-19': [9.29],
 'Kepler-2': [2.2],
 'Kepler-21': [2.79],
 'Kepler-23': [7.1, 15.27],
 'Kepler-27': [15.33, 31.33],
 'Kepler-28': [5.91, 8.98],
 'Kepler-3': [4.89],
 'Kepler-4': [3.21],
 'Kepler-5': [3.58],
 'Kepler-6': [3.23],
 'Kepler-7': [4.89],
 'Kepler-8': [3.522]}

In [176]:
grouped_neg = group(neg)
grouped_neg

{'KIC 10000785': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000797': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000800': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000823': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000827': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000876': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000939': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000941': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000962': [1.0, 6.0, 36.0, 85.0],
 'KIC 10000976': [1.0, 6.0, 36.0, 85.0],
 'Kepler-12': [13.3, 17.8, 22.2, 31.1],
 'Kepler-8': [7.05, 10.6, 14.1]}

## Doplnění negativních period

In [188]:
grouped = []

for host in grouped_pos:
    periods = grouped_pos[host]
    
    grouped_item = {"host": host, "periods": []}

    grouped_item["periods"] += [{"period": round(periods[-1] * 1.67, 3), "planet": 0}]
    grouped_item["periods"] += [{"period": round(periods[-1] * 2, 2), "planet": 0}]
    grouped_item["periods"] += [{"period": round(periods[-1] * 3, 3), "planet": 0}]
    grouped_item["periods"] += [{"period": round(periods[-1] * 4, 3), "planet": 0}]
    grouped_item["periods"] += [{"period": round(periods[-1] * 8, 3), "planet": 0}]

    for i in range(len(periods)):
        next = periods[i]
        current = {"period": round(next, 3), "planet": 1}

        if i > 0:
            last = periods[i - 1]
            mid = max((next + last) / 2, 0.6 * 1.13)
            grouped_item["periods"] += [{"period": round(mid, 3), "planet": 0}, current]
        else:
            grouped_item["periods"] += [current]

    grouped.append(grouped_item)

for host in grouped_neg:
    periods = grouped_neg[host]
    grouped_item = {"host": host, "periods": list(map(lambda p: {"period": p, "planet": 0}, periods))}
    grouped.append(grouped_item)

n_views = sum([len(host["periods"]) for host in grouped])

grouped, n_views

([{'host': 'Kepler-1',
   'periods': [{'period': 4.125, 'planet': 0},
    {'period': 4.94, 'planet': 0},
    {'period': 7.41, 'planet': 0},
    {'period': 9.88, 'planet': 0},
    {'period': 19.76, 'planet': 0},
    {'period': 2.47, 'planet': 1}]},
  {'host': 'Kepler-11',
   'periods': [{'period': 21.743, 'planet': 0},
    {'period': 26.04, 'planet': 0},
    {'period': 39.06, 'planet': 0},
    {'period': 52.08, 'planet': 0},
    {'period': 104.16, 'planet': 0},
    {'period': 32.0, 'planet': 1},
    {'period': 27.34, 'planet': 0},
    {'period': 22.68, 'planet': 1},
    {'period': 17.85, 'planet': 0},
    {'period': 13.02, 'planet': 1}]},
  {'host': 'Kepler-12',
   'periods': [{'period': 7.415, 'planet': 0},
    {'period': 8.88, 'planet': 0},
    {'period': 13.32, 'planet': 0},
    {'period': 17.76, 'planet': 0},
    {'period': 35.52, 'planet': 0},
    {'period': 4.44, 'planet': 1}]},
  {'host': 'Kepler-13',
   'periods': [{'period': 2.939, 'planet': 0},
    {'period': 3.52, 'planet': 0

# Trénovací množina

## Stažení TPS a uložení křivek

In [206]:
lcs = {}

for item in grouped:
    host, periods = item["host"], item["periods"]
    lc_path = f".cache/lc/{host}.fits"

    try:
        lc = lk.read(lc_path)
    except:
        result = lk.search_targetpixelfile(host, mission="Kepler", exptime="long")
        tps = result.download_all()
        lcc = map(lambda tp: tp.to_lightcurve(aperture_mask=tp.pipeline_mask).flatten(window_length=201), tps)
        lcc = lk.LightCurveCollection(lcc)
        lcc.plot()
        lc = lcc.stitch().remove_outliers(sigma_upper=3, sigma_lower=20)
        lc.to_fits(lc_path)

    lcs[host] = lc

## Hledání period a sestavení lokálních a globálních pohledů

In [207]:
def replace_nans(lc):
    flux = lc.flux.value

    last = np.nanmedian(flux) if np.isnan(flux[0]) else flux[0]
    delta_max = ((np.nanmax(flux) - np.nanmin(flux)) / 20)

    for i in range(len(flux)):
        if np.isnan(flux[i]):
            flux[i] = last + delta_max * np.random.rand()
        else:
            last = flux[i]

    np.nan_to_num(lc.flux_err.value, copy=False, nan=0, posinf=0, neginf=0)

    return lc

In [208]:
GV_SIZE = 2001
LV_SIZE = 201

lv_input, gv_input, output = np.zeros((n_views, LV_SIZE, 1)), np.zeros((n_views, GV_SIZE, 1)), np.zeros((n_views, 1))
i = 0

for item in grouped:
    lc = lcs[item["host"]]

    for period in item["periods"]:
        per = period["period"]
        delta = min(2, 0.05 * per)
        pdg = lc.to_periodogram("bls", period=np.linspace(max(0.5, per - delta), per + delta, 10000))
        per, dur, t0 = pdg.period_at_max_power, pdg.duration_at_max_power, pdg.transit_time_at_max_power
        fold = lc.fold(per, t0)

        gv = replace_nans(fold.bin(bins=GV_SIZE))
        gv = gv.normalize() - 1
        gv = (gv / np.abs(np.nanmin(gv.flux))) * 2.0 + 1
        
        fractional_duration = dur / per
        width = per.value * 1
        phase_mask = (fold.phase.value > -width * fractional_duration) & (fold.phase.value < width * fractional_duration)
        lc_zoom = fold[phase_mask]
        lv = replace_nans(lc_zoom.bin(bins=LV_SIZE))
    
        lv = lv.normalize() - 1
        lv = (lv / np.abs(np.nanmin(lv.flux))) * 2.0 + 1

        lv_input[i,:] = np.resize(lv.flux.value, lv_input[i].shape)
        gv_input[i,:] = np.resize(gv.flux.value, gv_input[i].shape)
        output[i,0] = period["planet"]

        if period["planet"]:
            mask = pdg.get_transit_mask(period=per, transit_time=t0, duration=dur)
            lc = lc[~mask].remove_nans()

        #lv.scatter()
        #gv.scatter()
        i += 1

`period` contains 105099 points.Periodogram is likely to be large, and slow to evaluate. Consider setting `frequency_factor` to a higher value.
`period` contains 175520 points.Periodogram is likely to be large, and slow to evaluate. Consider setting `frequency_factor` to a higher value.
`period` contains 147511 points.Periodogram is likely to be large, and slow to evaluate. Consider setting `frequency_factor` to a higher value.
`period` contains 123163 points.Periodogram is likely to be large, and slow to evaluate. Consider setting `frequency_factor` to a higher value.
`period` contains 246327 points.Periodogram is likely to be large, and slow to evaluate. Consider setting `frequency_factor` to a higher value.
`period` contains 172770 points.Periodogram is likely to be large, and slow to evaluate. Consider setting `frequency_factor` to a higher value.
`period` contains 144286 points.Periodogram is likely to be large, and slow to evaluate. Consider setting `frequency_factor` to a higher

# Neuronová sítě
## Formát vstupních dat

In [209]:
lv, gv, tg = np.copy(lv_input), np.copy(gv_input), np.copy(output)

n_lv, *lv_shape = lv.shape
n_gv, *gv_shape = gv.shape
n_targets, *target_shape = tg.shape

lv_shape, gv_shape, target_shape, n_lv

([201, 1], [2001, 1], [1], 179)

## Architektura

In [210]:
# LV branch.
lv_inp = Input(lv_shape)
lv_branch = Conv1D(16, (3), activation="relu")(lv_inp)
lv_branch = MaxPool1D()(lv_branch)
lv_branch = Conv1D(32, (3), activation="relu")(lv_branch)
lv_branch = MaxPool1D()(lv_branch)
lv_branch = Flatten()(lv_branch)
lv_branch = Model(lv_inp, lv_branch)

# GV branch.
gv_inp = Input(gv_shape)
gv_branch = Conv1D(16, (3), activation="relu")(gv_inp)
gv_branch = MaxPool1D()(gv_branch)
gv_branch = Conv1D(32, (3), activation="relu")(gv_branch)
gv_branch = MaxPool1D()(gv_branch)
gv_branch = Conv1D(64, (3), activation="relu")(gv_branch)
gv_branch = MaxPool1D()(gv_branch)
gv_branch = Conv1D(128, (3), activation="relu")(gv_branch)
gv_branch = MaxPool1D()(gv_branch)
gv_branch = Flatten()(gv_branch)
gv_branch = Model(gv_inp, gv_branch)

# Main branch.
main_branch = Concatenate()([lv_branch.output, gv_branch.output])
main_branch = Dense(128, activation="tanh")(main_branch)
main_branch = Dense(128, activation="tanh")(main_branch)
main_branch = Dropout()(main_branch)
main_branch = Dense(1, activation="tanh")(main_branch)
model = Model([lv_branch.input, gv_branch.input], main_branch)

model.compile(optimizer=RMSprop(), loss="mse")

model.summary()

Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 2001, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_62 (Conv1D)              (None, 1999, 16)     64          input_22[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_62 (MaxPooling1D) (None, 999, 16)      0           conv1d_62[0][0]                  
__________________________________________________________________________________________________
conv1d_63 (Conv1D)              (None, 997, 32)      1568        max_pooling1d_62[0][0]           
___________________________________________________________________________________________

## Trénovací a testovací množina

In [211]:
def shuffle(*args, seed=None):
    i = np.arange(args[0].shape[0])
    np.random.seed(seed)
    np.random.shuffle(i)
    return tuple(map(lambda x: x[i], args))

test_size = 2
#lv, gv, tg = shuffle(lv, gv, tg)

train_gv, test_gv = gv[:-test_size + 1], gv[-test_size + 1:]
train_lv, test_lv = lv[:-test_size + 1], lv[-test_size + 1:]
train_tg, test_tg = tg[:-test_size + 1], tg[-test_size + 1:]

train_gv, test_gv = gv[:], gv[:]
train_lv, test_lv = lv[:], lv[:]
train_tg, test_tg = tg[:], tg[:]

lv.shape, gv.shape, tg.shape
train_lv.shape, test_lv.shape, train_gv.shape, train_lv.shape

((179, 201, 1), (179, 201, 1), (179, 2001, 1), (179, 201, 1))

In [212]:
callback = EarlyStopping(monitor="loss", patience=50)
model.fit([train_lv, train_gv], train_tg, epochs=10000, validation_split=0.15, callbacks=[callback])

Epoch 1/10000
5/5 [==============================] - 1s 113ms/step - loss: 0.9459 - val_loss: 1.0000
Epoch 2/10000
5/5 [==============================] - 0s 83ms/step - loss: 1.5200 - val_loss: 1.0000
Epoch 3/10000
5/5 [==============================] - 0s 82ms/step - loss: 1.4419 - val_loss: 1.0000
Epoch 4/10000
5/5 [==============================] - 0s 84ms/step - loss: 1.5655 - val_loss: 0.9999
Epoch 5/10000
5/5 [==============================] - 0s 85ms/step - loss: 1.5704 - val_loss: 0.7365
Epoch 6/10000
5/5 [==============================] - 0s 84ms/step - loss: 0.8517 - val_loss: 0.1953
Epoch 7/10000
5/5 [==============================] - 0s 85ms/step - loss: 0.2839 - val_loss: 0.6360
Epoch 8/10000
5/5 [==============================] - 0s 92ms/step - loss: 0.3860 - val_loss: 0.0620
Epoch 9/10000
5/5 [==============================] - 0s 91ms/step - loss: 0.2871 - val_loss: 3.9742e-04
Epoch 10/10000
5/5 [==============================] - 0s 86ms/step - loss: 0.1897 - val_loss: 0

## Testování

In [213]:
results = np.abs(np.round(model.predict([test_lv, test_gv])))
errors = (results != test_tg).sum()
percentage = 100 - 100 * np.round(errors / len(test_lv), 2)

f"Errors: {errors} (success rate: {percentage} %)"

'Errors: 0 (success rate: 100.0 %)'

In [214]:
model.save("transit_nn.h5")

(173, 201, 1)